In [9]:
# '''
# function ConnectButton(){
#     console.log("Connect pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }

# setInterval(ConnectButton,60000);
# '''

# from google.colab import drive
# from os import chdir

# drive.mount('/content/drive')
# project_path = '/content/drive/MyDrive/Gproject/o-linked-site-prediction-feature-augment'
# chdir(project_path)

In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# set seed for the reproducible result
SEED = 42

In [11]:
data_dir = './data/integrated_features' # we will get names from the augmented proteins
protein_names = [x.split('.')[0] for x in os.listdir(data_dir) if x.split('.')[1] == 'csv'] # get protein name list to be processed for building machine learning models
print('the number of initial proteins:', len(protein_names))
print(protein_names[:10])

the number of initial proteins: 104
['A2ABU4', 'A2AHJ4', 'A2AKB9', 'A2AQ25', 'E9Q1P8', 'E9Q5G3', 'O08537', 'O09061', 'O35303', 'O70263']


## hyper parameter optimization by K-fold cross-validation

In [12]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold

import time
from IPython.display import clear_output

from ml_models import *

epochs = 1000
from keras.callbacks import EarlyStopping
patience = 30
callbacks = [EarlyStopping(patience=patience, restore_best_weights=True, monitor='val_loss')]

test_size = 0.2


### set initial parameters

In [13]:
from functions import *

In [14]:
initial_params = default_params.copy()
initial_params.update({
    'window_size'    : 10,
    'rnn_layers'     : 4,
    'rnn_neurons'    : 64,
    'dnn_layers'     : 3,
    'dnn_neurons'    : 64
    })

print('initial parameters')
for key, value in initial_params.items():
    print(f'{key:<14} : {value}')

search_space = {
    'regularizer'    : [
        {'input': 'L21_0.001',   'hidden': None,          'bias': None},
        {'input': 'L21_0.001',   'hidden': 'L21_0.001',   'bias': None},
        {'input': 'L21_0.0001',  'hidden': None,          'bias': None},
        {'input': 'L21_0.0001',  'hidden': 'L21_0.0001',  'bias': None},
        {'input': 'L21_0.00001', 'hidden': None,          'bias': None},
        {'input': 'L21_0.00001', 'hidden': 'L21_0.00001', 'bias': None},
        {'input': 'L21_0.000001', 'hidden': None,          'bias': None},
        {'input': 'L21_0.000001', 'hidden': 'L21_0.000001', 'bias': None},

        {'input': 'L1_0.001',   'hidden': None,         'bias': None},
        {'input': 'L1_0.001',   'hidden': 'L1_0.001',   'bias': None},
        {'input': 'L1_0.0001',  'hidden': None,         'bias': None},
        {'input': 'L1_0.0001',  'hidden': 'L1_0.0001',  'bias': None},
        {'input': 'L1_0.00001', 'hidden': None,         'bias': None},
        {'input': 'L1_0.00001', 'hidden': 'L1_0.00001', 'bias': None},
        {'input': 'L1_0.000001', 'hidden': None,         'bias': None},
        {'input': 'L1_0.000001', 'hidden': 'L1_0.000001', 'bias': None},
        ],
}

initial parameters
rnn_layers     : 4
rnn_neurons    : 64
dnn_layers     : 3
dnn_neurons    : 64
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


## set variables

In [15]:
model_type = 'SLSTM_UP_AUGMENT'

augmented_columns = dict(pd.read_csv('./data/augmented_columns.csv', header=0).values.squeeze())
print('# of augmented features:', len(augmented_columns))

# set continuous input features
x_cts = [x for x in augmented_columns.keys() if augmented_columns.get(x) != 'object']
print('continuous features:')
display(dict(zip(range(len(x_cts)), x_cts)))

# set categorical input features
x_cat = ['residue'] + \
        [x for x in augmented_columns.keys() if augmented_columns.get(x) == 'object']
print('categorical features:')
print(dict(zip(range(len(x_cat)), x_cat)))

# input features
x_var = x_cts + x_cat

# set continuos output feature
y_cts = []

# set categorical output feature
y_cat = ['positivity']

# output features
y_var = y_cts + y_cat

# of augmented features: 498
continuous features:


{0: 'number_hydrophobic_0A',
 1: 'number_hydrophilic_0A',
 2: 'number_polar_0A',
 3: 'number_aromatic_0A',
 4: 'number_aliphatic_0A',
 5: 'number_charged_0A',
 6: 'number_positive_0A',
 7: 'number_negative_0A',
 8: 'number_gly_0A',
 9: 'number_very_small_0A',
 10: 'number_small_0A',
 11: 'number_normal_0A',
 12: 'number_long_0A',
 13: 'number_pro_0A',
 14: 'number_A_polar_uncharged_with_hydroxyl_group_0A',
 15: 'number_b_polar_uncharged_with_amide_0A',
 16: 'number_d_negatively_charged_polar_0A',
 17: 'number_e_non_polar_suffered_0A',
 18: 'number_f_non_polar_aromatic_0A',
 19: 'number_ala_0A',
 20: 'number_cys_0A',
 21: 'number_asp_0A',
 22: 'number_glu_0A',
 23: 'number_phe_0A',
 24: 'number_his_0A',
 25: 'number_ile_0A',
 26: 'number_lys_0A',
 27: 'number_leu_0A',
 28: 'number_met_0A',
 29: 'number_asn_0A',
 30: 'number_gln_0A',
 31: 'number_arg_0A',
 32: 'number_ser_0A',
 33: 'number_thr_0A',
 34: 'number_val_0A',
 35: 'number_trp_0A',
 36: 'number_tyr_0A',
 37: 'sasa_hydrophobic_0

categorical features:
{0: 'residue'}


# build amino acid sequence dataset

### model training with K-fold

In [16]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update  = False

params = initial_params.copy()
MODELs = []
METRICs = []
METRIC_MEAN = []
feature_weights = []

model_id = 1
verbose = 0
for param_name, space in search_space.items():
    for point in space:
        clear_output(wait=True)
        display(METRIC_MEAN)
        params[param_name] = point
        for key, value in search_space.items():
            print(f'{key:<14} : {value}')
        
        data_x = []
        data_y = []
        feature_weight = []

        for name in protein_names:
            data = pd.read_csv(f'./data/integrated_features/{name}.csv')
            ST_idx = np.where((data['residue'] == 'S') | (data['residue'] == 'T'))[0]
            
            # get X dataset
            x_onehot = get_onehots(data[x_var], columns = x_cat)
            x_features = list(x_onehot.columns)
            
            # get Y dataset
            y_onehot = get_onehots(data[y_var], columns = y_cat)
            y_labels = list(y_onehot.columns)
            
            for idx in ST_idx:
                window_x = np.array(get_window(x_onehot, idx, params['window_size']))
                label_y  = np.array(y_onehot.iloc[idx])
                
                data_x.append(window_x)
                data_y.append(label_y)
                
        data_x = np.array(data_x)
        data_y = np.array(data_y)
        model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) # I don't know why, but this row is helping producing the same training result of a neural network

        print('data x shape:', data_x.shape)
        print('data y shape:', data_y.shape)
        print('class y counts:', data_y.sum(0))
        print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')
        
        splitter = StratifiedShuffleSplit(n_splits = 1, test_size = test_size, random_state = SEED)
        train_idx, test_idx = list(splitter.split(data_x, data_y))[0]
        
        train_x = data_x[train_idx]
        train_y = data_y[train_idx]
        
        test_x = data_x[test_idx]
        test_y = data_y[test_idx]
        
        train_x, test_x = data_scaling(train_x, test_x)
        
        splitter_kf = KFold(n_splits = 5)
        for cv_idx, (train_idx_kf, test_idx_kf) in enumerate(splitter_kf.split(train_x, train_y)):
            train_x_kf, train_y_kf = train_x[train_idx_kf], train_y[train_idx_kf]
            test_x_kf, test_y_kf   = train_x[test_idx_kf],  train_y[test_idx_kf]
            
            # up-sample the training dataset
            train_x_kf, train_y_kf = upsample_data(train_x_kf, train_y_kf) 
            
            model = LSTM_CLS(data_x.shape[1], data_x.shape[-1], data_y.shape[-1], params) 
            model_name = name_model(f'{model_type}_KFOLD', params)
            
            model_folder  = f'./models/{model_name}_{data_x.shape}'
            if not os.path.exists(model_folder):
                os.makedirs(model_folder)
            model_path    = f'{model_folder}/{cv_idx}.h5'
            metric_path   = f'{model_folder}/{cv_idx}.csv'
            
            
            if not os.path.exists(model_path) or model_update:
                time_start = time.time()
                history = model.fit(train_x_kf, train_y_kf, verbose=verbose, 
                                    epochs = 10000, callbacks = callbacks,
                                    validation_data = (test_x_kf, test_y_kf))
                time_end = time.time()
                training_time = round((time_end - time_start)/60, 3)
                
                model.save_weights(model_path)
                
                test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
                model_metrics = {
                    'model_id' : model_id,
                    'cv_idx'   : cv_idx,
                    **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
                    'train_y'     : train_y.shape[-1],
                    'test_size'   : test_x.shape[0],
                    **params,
                    'regularizer_input' : params['regularizer']['input'],
                    'regularizer_hidden' : params['regularizer']['hidden'],
                    'regularizer_bias' : params['regularizer']['bias'],
                    'training_time': training_time,
                    'test_loss': test_loss,
                    'accuracy': accuracy,
                    **{f'precision_{x}': precision[x] for x in range(len(precision))},
                    **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
                    **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
                
                model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
                model_metrics.to_csv(metric_path, index=False)
                
            else:
                model.load_weights(model_path)
                model_metrics = pd.read_csv(metric_path, header=0)
                
            print(f'f1 score: {model_metrics.f1_1[0]}')
            
            model_metrics['model_id'] = model_id
            METRICs.append(model_metrics)
            MODELs.append(model)
            weights = model.get_weights()
            feature_weight.append(np.abs(weights[0].mean(1)).reshape(1,-1))
        
        METRIC_MEAN = pd.concat(METRICs).groupby('model_id').mean()
        f1_best = METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0].f1_1
        print(f'best f1 score: {f1_best}')
        model_id += 1
        feature_weight = np.concatenate(feature_weight, axis=0)
        feature_weight = feature_weight.mean(0)
        feature_weight = pd.Series(feature_weight, index=x_features)
        feature_weight.to_csv(f'./weights/{model_name}_{data_x.shape}.csv')
        feature_weights.append(feature_weight)

    # if param_name in ['learning_rate']: # for float-type parameters
    #     best_value = float(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
    #     params[param_name] = best_value
    #     search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"

    # else: # for int-type parameters
    #     best_value = int(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
    #     params[param_name] = best_value
    #     search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"
    

clear_output(wait=True)
display(METRIC_MEAN)
for key, value in search_space.items():
    print(f'{key:<14} : {value}')


,cv_idx,train_0,train_1,train_2,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,5.0796,0.486763,86.004,97.300,11.498,87.956,35.582,92.168,14.942
2,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,6.3096,0.708349,96.270,96.270,0.000,100.000,0.000,98.100,0.000
3,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,9.0076,0.385164,94.518,97.008,24.498,97.306,22.558,97.156,23.170
4,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,6.1848,0.470403,90.320,97.144,17.148,92.676,29.534,94.788,19.996
5,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,5.0282,0.327553,94.564,97.182,28.102,97.174,27.212,97.174,27.068
6,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,6.7376,0.351533,94.814,97.042,27.240,97.584,23.258,97.316,25.060
7,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,5.6174,0.420123,91.040,97.028,15.454,93.568,25.812,95.212,18.210
8,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,5.2542,0.487802,84.832,97.458,11.550,86.514,41.396,91.578,17.208
9,2.0,9222.0,21.0,518.0,2.0,2306.0,4.0,64.0,3.0,64.0,...,NaN,6.2040,0.332221,94.830,97.170,29.588,97.468,26.744,97.318,27.864


regularizer    : [{'input': 'L21_0.001', 'hidden': None, 'bias': None}, {'input': 'L21_0.001', 'hidden': 'L21_0.001', 'bias': None}, {'input': 'L21_0.0001', 'hidden': None, 'bias': None}, {'input': 'L21_0.0001', 'hidden': 'L21_0.0001', 'bias': None}, {'input': 'L21_0.00001', 'hidden': None, 'bias': None}, {'input': 'L21_0.00001', 'hidden': 'L21_0.00001', 'bias': None}, {'input': 'L1_0.001', 'hidden': None, 'bias': None}, {'input': 'L1_0.001', 'hidden': 'L1_0.001', 'bias': None}, {'input': 'L1_0.0001', 'hidden': None, 'bias': None}, {'input': 'L1_0.0001', 'hidden': 'L1_0.0001', 'bias': None}, {'input': 'L1_0.00001', 'hidden': None, 'bias': None}, {'input': 'L1_0.00001', 'hidden': 'L1_0.00001', 'bias': None}, {'input': 'L2_0.001', 'hidden': None, 'bias': None}, {'input': 'L2_0.001', 'hidden': 'L2_0.001', 'bias': None}, {'input': 'L2_0.0001', 'hidden': None, 'bias': None}, {'input': 'L2_0.0001', 'hidden': 'L2_0.0001', 'bias': None}, {'input': 'L2_0.00001', 'hidden': None, 'bias': None}, {

KeyboardInterrupt: 

In [ ]:
print('data x shape: ', data_x.shape)
print('data y shape: ', data_y.shape)
print('train x shape:', train_x.shape)
print('test  x shape:', test_y.shape)

data x shape:  (12120, 420)
data y shape:  (12120, 2)
train x shape: (9696, 420)
test  x shape: (2424, 2)


In [ ]:
idx = 0
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 1
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 2
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 3
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 4
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 5
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 6
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 7
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 8
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 9
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 10
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 11
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


In [ ]:
idx = 15
weights = feature_weights[idx]
n_features = len(x_features)
plt.figure(figsize=(n_features/2, n_features/2*9/16))
plt.grid(axis='both', color = "grey", linewidth = "0.1", linestyle = "--", zorder=0)

plt.bar(range(len(weights)), weights, align='center')
plt.xticks(ticks=range(n_features), labels=x_features, rotation='vertical')

plt.show()

optimal parameters
dnn_layers     : 4
dnn_neurons    : 256
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10
